In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn.model_selection
import sklearn.preprocessing
import string
from keras.layers import TextVectorization
from bs4 import BeautifulSoup
import requests
from joblib import Parallel,delayed
import math

2024-08-08 17:51:07.652056: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/trannhi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
website_categories = pd.read_csv('website_categories.csv')

In [7]:
website_categories

,Unnamed: 0,url,category,title
0,7,touristscavengerhunt.com,/Travel,explore the city with our 2-3 hour tourist sca...
1,11,plannedspontaneityhiking.com,/Hobbies & Leisure/Outdoors/Hiking & Camping,a blog about hiking around the midwest and beyond
2,19,www.utvdirect.com,/Autos & Vehicles/Motor Vehicles (By Type)/Off...,find the right side by side accessories for yo...
3,20,www.debrovys.com,/Business & Industrial/Construction & Maintena...,debrovy's is the ideal tarp manufacturer in th...
4,23,www.urbanbarn.com,/Home & Garden/Home Furnishings,furniture and accessories for your home and of...
...,...,...,...,...
27059,98999,historyfiles.co.uk,/Reference/Geographic Reference,an extensive collection of information coverin...
27060,99000,therugbynetwork.com,/Sports/Team Sports/Rugby,watch every game of major league rugby live an...
27061,99001,theseniorlist.com,/People & Society/Seniors & Retirement,the senior list offers the best advice for sen...
27062,99003,mypetchicken.com,/Business & Industrial/Agriculture & Forestry/...,backyard chickens from my pet chicken: offerin...


In [8]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because",
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here",
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should",
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves"]

In [10]:
# preprocessing functions
# remove stopwords from titles
def remove_stopwords(data):
  data['title'] = data['title'].apply(lambda x : ' '.join([word for word in str(x).split() if word not in (stopwords)]))

In [11]:
# take away all punctuation 
def remove_punctuation(test_str):
# Using filter() and lambda function to filter out punctuation characters
  result = ''.join(filter(lambda x: x.isalpha() or x.isdigit() or x.isspace(), test_str))
  return result

In [12]:
# take the first general category only
def reduce_labels(label):
  ans = ''
  for char in str(label)[1:]:
    if char=='/':
      return ans
    else:
      ans+=char
  return ans

In [13]:
# removes all the stopwords from our csv file
remove_stopwords(website_categories)
# apply the reduce_labels function to each element in the category column
website_categories['category'] = website_categories['category'].apply(reduce_labels)
# apply the remove_punctuation to each element in the title column
website_categories['title'] = website_categories['title'].apply(remove_punctuation)

In [14]:
# get a list of unique categories
categories = website_categories['category'].unique()
# enumerate the categories by index and map the category to its index in the dictionary
class_names = {}
for i,category in enumerate(categories):
  class_names[category] = i
class_names

{'Travel': 0,
 'Hobbies & Leisure': 1,
 'Autos & Vehicles': 2,
 'Business & Industrial': 3,
 'Home & Garden': 4,
 'Online Communities': 5,
 'Arts & Entertainment': 6,
 'Shopping': 7,
 'People & Society': 8,
 'Reference': 9,
 'Beauty & Fitness': 10,
 'Law & Government': 11,
 'Health': 12,
 'Books & Literature': 13,
 'News': 14,
 'Games': 15,
 'Food & Drink': 16,
 'Adult': 17,
 'Finance': 18,
 'Jobs & Education': 19,
 'Internet & Telecom': 20,
 'Sports': 21,
 'Real Estate': 22,
 'Computers & Electronics': 23,
 'Science': 24,
 'Sensitive Subjects': 25,
 'Pets & Animals': 26,
 'an': 27,
 'Business and Consumer Services': 28,
 'Computers Electronics and Technology': 29,
 'eCommerce & Shopping': 30,
 'Science and Education': 31,
 'Reference Materials': 32,
 'Vehicles': 33,
 'Law and Government': 34,
 'Food and Drink': 35,
 'Travel and Tourism': 36,
 'Community and Society': 37,
 'Gambling': 38,
 'Lifestyle': 39,
 'Hobbies and Leisure': 40,
 'Home and Garden': 41,
 'Heavy Industry and Enginee

In [15]:
def apply_encodings(category):
  return class_names[category]
# count the occurrences of each unique value in the category column in descending order
website_categories['category'].value_counts()

category
Business & Industrial                   3005
Arts & Entertainment                    2412
Shopping                                2338
Internet & Telecom                      2049
Hobbies & Leisure                       1683
Computers & Electronics                 1429
People & Society                        1323
Home & Garden                           1276
Health                                  1014
Online Communities                       920
Sports                                   893
Jobs & Education                         890
Autos & Vehicles                         866
Food & Drink                             781
News                                     773
Beauty & Fitness                         647
Games                                    640
Travel                                   630
Adult                                    529
Law & Government                         513
Books & Literature                       510
Finance                                  443
R

In [61]:
# inverse map from encodings to classnames
inv_map = {v: k for k, v in class_names.items()}
inv_map

{0: 'Travel',
 1: 'Hobbies & Leisure',
 2: 'Autos & Vehicles',
 3: 'Business & Industrial',
 4: 'Home & Garden',
 5: 'Online Communities',
 6: 'Arts & Entertainment',
 7: 'Shopping',
 8: 'People & Society',
 9: 'Reference',
 10: 'Beauty & Fitness',
 11: 'Law & Government',
 12: 'Health',
 13: 'Books & Literature',
 14: 'News',
 15: 'Games',
 16: 'Food & Drink',
 17: 'Adult',
 18: 'Finance',
 19: 'Jobs & Education',
 20: 'Internet & Telecom',
 21: 'Sports',
 22: 'Real Estate',
 23: 'Computers & Electronics',
 24: 'Science',
 25: 'Sensitive Subjects',
 26: 'Pets & Animals',
 27: 'an',
 28: 'Business and Consumer Services',
 29: 'Computers Electronics and Technology',
 30: 'eCommerce & Shopping',
 31: 'Science and Education',
 32: 'Reference Materials',
 33: 'Vehicles',
 34: 'Law and Government',
 35: 'Food and Drink',
 36: 'Travel and Tourism',
 37: 'Community and Society',
 38: 'Gambling',
 39: 'Lifestyle',
 40: 'Hobbies and Leisure',
 41: 'Home and Garden',
 42: 'Heavy Industry and Eng

In [19]:
value_counts = pd.read_csv('value_counts.csv')
value_counts

,Unnamed: 0,category
0,/Internet & Telecom/Web Services/Web Design & ...,1043
1,/Business & Industrial,992
2,/People & Society/Religion & Belief,666
3,/Internet & Telecom,590
4,/News,575
...,...,...
555,/Science/Physics,1
556,/Sports/Boxing,1
557,/Finance/Investing/Stocks & Bonds,1
558,/Food & Drink/Restaurants/Restaurant Reviews &...,1


In [20]:
top_9_categories = ['Business & Industrial','Arts & Entertainment','Shopping','Internet & Telecom','Hobbies & Leisure','Computers & Electronics','People & Society','Home & Garden','Health']

In [21]:
# only include rows where the category column's value is in a specified list of top categories.
top_9 = website_categories.loc[website_categories['category'].isin(top_9_categories)]

In [22]:
# keep only the category and title columns
top_9 = top_9[['category','title']]

In [23]:
max_length = 1013
# a dictionary to keep track of the number of titles added for each category.
top_9_counts = {}
category = []
title = []

In [24]:
for index, row in top_9.iterrows():
  # if the category is not in the dictionary, initialize it with a count of 0 and add the title and category to the lists
  if row['category'] not in top_9_counts.keys():
    top_9_counts[row['category']] = 0
    title.append(row['title'])
    category.append(row['category'])

  elif top_9_counts[row['category']]<max_length:
    top_9_counts[row['category']]+=1
    title.append(row['title'])
    category.append(row['category'])

final_top_9 = pd.DataFrame()
final_top_9['category'] = category
final_top_9['title'] = title

# apply the encodings the final list
encoded_categories = final_top_9['category'].apply(apply_encodings)

In [25]:
X_train, X_test,Y_train, Y_test = sklearn.model_selection.train_test_split(final_top_9['title'], encoded_categories, test_size=0.2, random_state = 45)
# initializes a tokenizer that will consider the top 50,000 most frequent words from text data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 50000)
# fits the tokenizer on the training text data, and builds the vocabulary based on the training data
tokenizer.fit_on_texts(X_train)
# generates a dictionary (word_index) where each word in the vocabulary is mapped to an integer index
words_to_index = tokenizer.word_index

In [27]:
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip -q glove.6B.zip

--2024-08-08 18:21:21--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M   518KB/s    in 14m 47s 

2024-08-08 18:36:08 (950 KB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [28]:
# use Glove Embeddings
path_to_glove_file = 'glove.6B.50d.txt'

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [29]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=25)
vectorizer.adapt(final_top_9['title'])
x_train = vectorizer(np.array([[s] for s in X_train])).numpy()
x_val = vectorizer(np.array([[s] for s in X_test])).numpy()
y_train = np.array(Y_train)
y_val = np.array(Y_test)


In [30]:
num_tokens = len(words_to_index) + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in words_to_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 16423 words (7111 misses)


In [31]:
embedding_layer = tf.keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")

In [32]:
embedded_sequences = embedding_layer(int_sequences_input)
x = tf.keras.layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv1D(128, 5, activation="relu")(x)
x = tf.keras.layers.Dropout(.2)(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
x = tf.keras.layers.Dense(100, activation="relu")(x)

preds = tf.keras.layers.Dense(len(class_names), activation="softmax")(x)
model = tf.keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 50)          1176800   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         32128     
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0     

In [33]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.001), metrics=["accuracy"]
)

In [40]:
model.fit(x_train, y_train, batch_size=128, epochs=40, validation_data=(x_val, y_val))

Epoch 1/40
58/58 [==============================] - 2s 40ms/step - loss: 0.2779 - accuracy: 0.9053 - val_loss: 3.1221 - val_accuracy: 0.3549
Epoch 2/40
58/58 [==============================] - 2s 35ms/step - loss: 0.3077 - accuracy: 0.8977 - val_loss: 3.0566 - val_accuracy: 0.3505
Epoch 3/40
58/58 [==============================] - 2s 38ms/step - loss: 0.2832 - accuracy: 0.9062 - val_loss: 3.1113 - val_accuracy: 0.3456
Epoch 4/40
58/58 [==============================] - 2s 35ms/step - loss: 0.2803 - accuracy: 0.9060 - val_loss: 3.1993 - val_accuracy: 0.3330
Epoch 5/40
58/58 [==============================] - 2s 37ms/step - loss: 0.3284 - accuracy: 0.8890 - val_loss: 3.1045 - val_accuracy: 0.3467
Epoch 6/40
58/58 [==============================] - 2s 36ms/step - loss: 0.3757 - accuracy: 0.8767 - val_loss: 3.1312 - val_accuracy: 0.3483
Epoch 7/40
58/58 [==============================] - 2s 37ms/step - loss: 0.3005 - accuracy: 0.8971 - val_loss: 3.0736 - val_accuracy: 0.3510
Epoch 8/40
58

In [41]:
def predict_using_model(model, phrase, inv_map):
  string_input = keras.Input(shape=(1,), dtype="string")
  x = vectorizer(string_input)
  preds = model(x)
  end_to_end_model = keras.Model(string_input, preds)

  probabilities = end_to_end_model.predict(
    [[phrase]]
  )
  high = np.argmax(probabilities)
  return inv_map[high]

In [42]:
predict_using_model(model, 'friends', inv_map)

1/1 [==============================] - 0s 274ms/step


'People & Society'

In [54]:
def returntitle(url):
  try:
    request = requests.get(url)
  except:
    return 'error'
  Soup = BeautifulSoup(request.text, 'html.parser')
  title_tag = Soup.find('title')
  if title_tag:
    title_text = title_tag.text
  return title_text

In [55]:
def categorize(url):
  title = returntitle(url)
  category = predict_using_model(model, title, inv_map)
  return category

In [56]:
categorize('https://myanimelist.net/')

1/1 [==============================] - 0s 153ms/step


'People & Society'

In [57]:
categorize('https://github.com/tnhi26990?tab=repositories')

1/1 [==============================] - 0s 184ms/step


'Internet & Telecom'

In [58]:
categorize('https://www.youtube.com/')

1/1 [==============================] - 0s 272ms/step


'Home & Garden'

In [60]:
categorize('https://www.netflix.com/')

1/1 [==============================] - 0s 164ms/step


'Arts & Entertainment'